In this notebook, I run one step for TwoStreamControlModel to get intermediate outputs which I can then compare to my local run

In [1]:
!apt-get update -qq
!apt-get install -y libgl1-mesa-glx -qq

debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package libelf1:amd64.
(Reading database ... 30662 files and directories currently installed.)
Preparing to unpack .../00-libelf1_0.176-1.1ubuntu0.1_amd64.deb ...
Unpacking libelf1:amd64 (0.176-1.1ubuntu0.1) ...
Selecting previously unselected package libdrm-common.
Preparing to unpack .../01-libdrm-common_2.4.107-8ubuntu1~20.04.2_all.deb ...
Unpacking libdrm-common (2.4.107-8ubuntu1~20.04.2) ...
Selecting previously unselected package libdrm2:amd64.
Preparing to unpack .../02-libdrm2_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
Unpacking libdrm2:amd64 (2.4.107-8ubuntu1~20.04.2) ...
Selecting previously unselected package libdrm-amdgpu1:amd64.
Preparing to unpack .../03-libdrm-amdgpu1_2.4.107-8ubuntu1~20.04.2_amd64.deb ...
Unpacking libdrm-amdgpu1:amd64 (2.4.107-8ubuntu1~20.04.2) ...
Selecting previously unselected package libpciaccess0:amd64.
Preparing to unpack .../04-libpciaccess0_0.1

In [2]:
!git clone https://github.com/vislearn/ControlNet-XS -qq

fatal: destination path 'ControlNet-XS' already exists and is not an empty directory.


In [3]:
!rm -rf diffusers
!git clone https://github.com/UmerHA/diffusers.git --branch controlnet-xs -qq

In [4]:
!pip install -r ControlNet-XS/requirements/pt2.txt -qq
!pip install -e ControlNet-XS -qq

In [5]:
!pip install -Uqq transformers
!pip install diffusers -e diffusers -qq

In [6]:
assert False

AssertionError: 

___

Restart kernel here, so newly installed packages are available

___

In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'  # needed to make torch deterministic

In [2]:
import scripts.control_utils as cu
import torch
from PIL import Image

/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:114: UserWarning: Mapping deprecated model name vit_base_resnet50_384 to current vit_base_r50_s16_384.orig_in21k_ft_in1k.
  model = create_fn(


In [3]:
torch.use_deterministic_algorithms(True)

In [4]:
path_to_config = 'ControlNet-XS/configs/inference/sdxl/sdxl_encD_canny_48m.yaml'

If this results in the kernel crashing, I'm using too much GPU memory elsewhere. Shut down every other kernel and try again.

In [5]:
model = cu.create_model(path_to_config).to('cuda')

Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 320, out-chn: 320, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
constructing SpatialTransformer of depth 2 w/ 640 channels and 10 heads
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
Building a Downsample layer with 2 dims.
  --> settings are: 
 in-chn: 640, out-chn: 640, kernel-size: 3, stride: 2, padding: 1
constructing SpatialTransformer of depth 10 w/ 1280 channels and 20 heads
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using checkpointing
BasicTransformerBlock is using 

In [6]:
size = 768
num_samples=1
prompt='cinematic, shoe in the streets, made from meat, photorealistic shoe, highly detailed'
n_prompt='lowres, bad anatomy, worst quality, low quality'

In [7]:
def get_canny_edges():
    image_path = 'input_images/shoe.png' # chosen to fit size above
    image = cu.get_image(image_path, size=size)
    edges = cu.get_canny_edges(image, low_th=100, high_th=250)
    return edges
edges = get_canny_edges()

In [8]:
model.model.DEBUG_LOG_by_Umer = True

In [9]:
samples, controls, latents = cu.get_sdxl_sample(
    guidance=edges,
    ddim_steps=10,
    num_samples=num_samples,
    model=model,
    shape=[4, size // 8, size // 8],
    control_scale=0.95,
    prompt=prompt,
    n_prompt=n_prompt,
    return_latents=True
)

Global seed set to 1999158951


[CONTROL CORRECTION OF ControlledDiffusionEngine SCALED WITH 0.95]
>>> These are the embedders that are hard at working:
Embedder FrozenCLIPEmbedder       , output 0: Initialized  crossattn. It has now shape [1, 77, 768]
--
Embedder FrozenOpenCLIPEmbedder2  , output 0: Concatted to crossattn. It has now shape [1, 77, 2048]
Embedder FrozenOpenCLIPEmbedder2  , output 1: Initialized  vector   . It has now shape [1, 1280]
--
Where are inside ConcatTimestepEmbedderND, lets forward
- x.shape before rearrage = [1, 2]
- x.shape after rearrange = [2]
- emb.shape before rearrange = [2, 256]
- emb.shape after rearrange = [1, 512]
Embedder ConcatTimestepEmbedderND , output 0: Concatted to vector   . It has now shape [1, 1792]
--
Where are inside ConcatTimestepEmbedderND, lets forward
- x.shape before rearrage = [1, 2]
- x.shape after rearrange = [2]
- emb.shape before rearrange = [2, 256]
- emb.shape after rearrange = [1, 512]
Embedder ConcatTimestepEmbedderND , output 0: Concatted to vector   . I

RuntimeError: Debug Log saved successfully

Cool

In [10]:
def cls_name(o): return str(type(o)).replace("< class'","").replace("'>","").split('.')[-1]

for i,m in enumerate(model.conditioner.embedders):
    k = m.input_key if hasattr(m,'input_key') else m.input_keys
    print(f"{i}] {cls_name(m):<25}     - input key(s) = {k}")

0] FrozenCLIPEmbedder            - input key(s) = caption
1] FrozenOpenCLIPEmbedder2       - input key(s) = caption
2] ConcatTimestepEmbedderND      - input key(s) = original_size_as_tuple
3] ConcatTimestepEmbedderND      - input key(s) = crop_coords_top_left
4] ConcatTimestepEmbedderND      - input key(s) = target_size_as_tuple


In [11]:
model.conditioner.embedders[2]

ConcatTimestepEmbedderND(
  (timestep): Timestep()
)

In [12]:
model.conditioner.embedders[3]

ConcatTimestepEmbedderND(
  (timestep): Timestep()
)

In [13]:
model.conditioner.embedders[4]

ConcatTimestepEmbedderND(
  (timestep): Timestep()
)

In [17]:
conditional_vector_szs = (1280,1792,2304,2816)
for i in range(3):
    print(f'Dim of ConcatTimestepEmbedderND at {2+i}: ',end='')
    print(conditional_vector_szs[i+1]-conditional_vector_szs[i])

Dim of ConcatTimestepEmbedderND at 2: 512
Dim of ConcatTimestepEmbedderND at 3: 512
Dim of ConcatTimestepEmbedderND at 4: 512


In [9]:
print(model.model.diffusion_model.input_blocks[1][0].use_scale_shift_norm)
print(model.model.learn_embedding)
print(model.model.control_scale)
print(model.model.control_model.time_embed)

False
True
1.0
Sequential(
  (0): Linear(in_features=320, out_features=1280, bias=True)
  (1): SiLU()
  (2): Linear(in_features=1280, out_features=1280, bias=True)
)


Create guidance info

In [32]:
my_guidance = edges

my_ds = {
    'hint': my_guidance,
    'crop_coords_top_left': torch.tensor([0, 0]),
    'original_size_as_tuple': torch.tensor(my_guidance.shape[-2:]),
    'target_size_as_tuple': torch.tensor(my_guidance.shape[-2:]),
}

for k,v in my_ds.items(): print(k,'->',list(v.shape))

hint -> [768, 768]
crop_coords_top_left -> [2]
original_size_as_tuple -> [2]
target_size_as_tuple -> [2]


In addition to the `hint` image, SDXL was also conditionned on `crop_coords_top_left`, `original_size_as_tuple`, `target_size_as_tuple`

Make them batch-ready

In [33]:
my_batch = cu.get_batch(ds_instance=my_ds, num_samples=num_samples)

In [38]:
for k,v in my_batch.items(): print(k,'->',list(v.shape))

hint -> [1, 768, 768]
crop_coords_top_left -> [1, 2]
original_size_as_tuple -> [1, 2]
target_size_as_tuple -> [1, 2]


Add prompt

In [39]:
my_batch['caption'] = [prompt or my_ds['caption']] * num_samples

In [42]:
for k,v in my_batch.items(): print(k,'->',(list(v.shape) if hasattr(v,'shape') else len(v)))

hint -> [1, 768, 768]
crop_coords_top_left -> [1, 2]
original_size_as_tuple -> [1, 2]
target_size_as_tuple -> [1, 2]
caption -> 1


To cuda

In [44]:
for k in my_batch:
    if isinstance(my_batch[k], torch.Tensor):
        my_batch[k] = my_batch[k].to('cuda')

Same for negative prompt

In [45]:
import copy

In [49]:
my_batch_uc = copy.deepcopy(my_batch)
my_batch_uc['caption'] = [n_prompt] * num_samples

In [50]:
my_c, my_uc = model.conditioner.get_unconditional_conditioning(
    my_batch,
    batch_uc=my_batch_uc,
    force_uc_zero_embeddings=None if len(model.conditioner.embedders) > 0 else [],
)

In [51]:
my_c.keys()

dict_keys(['crossattn', 'vector'])

___

## Load local model

In [ ]:
import random
import numpy as np
import torch

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'mps'
device_dtype = torch.float16 if device == 'cuda' else torch.float32

In [ ]:
from diffusers import StableDiffusionXLPipeline
from diffusers import EulerDiscreteScheduler
from diffusers.models.controlnetxs import ControlNetXSModel
from diffusers.pipelines.controlnet_xs.pipeline_controlnet_xs_sd_xl import StableDiffusionXLControlNetXSPipeline

In [ ]:
sdxl_pipe = StableDiffusionXLPipeline.from_single_file('weights/sd_xl_base_1.0_0.9vae.safetensors').to(device)
cnxs = ControlNetXSModel.from_pretrained('weights/cnxs').to(device)

In [ ]:
cnxs.scale_list = cnxs.scale_list * 0. + 0.95
cnxs.base_model = sdxl_pipe.unet
sdxl_pipe.scheduler.config.timestep_spacing = 'linspace'
sdxl_pipe.scheduler = EulerDiscreteScheduler.from_config(sdxl_pipe.scheduler.config)

In [ ]:
cnxs_pipe = StableDiffusionXLControlNetXSPipeline(
    vae=sdxl_pipe.vae,
    text_encoder=sdxl_pipe.text_encoder,
    text_encoder_2=sdxl_pipe.text_encoder_2,
    tokenizer=sdxl_pipe.tokenizer,
    tokenizer_2=sdxl_pipe.tokenizer_2,
    unet=sdxl_pipe.unet,
    controlnet=cnxs,
    scheduler=sdxl_pipe.scheduler,
)

### Load intermediate outputs of whole model

In [ ]:
from util_inspect import load_intermediate_outputs

In [ ]:
model_outp_local = load_intermediate_outputs('intermediate_output/local_debug_log.pkl')
model_outp_cloud = load_intermediate_outputs('intermediate_output/cloud_debug_log.pkl')

### Load intermediate outputs of 1st resnets